This notebook is what we use to visualize attention on some test images by using the torchcam library.


In [1]:
from dataset.phishing_dataset import PhishingDataset

import torch
from torchcam.methods import SmoothGradCAMpp
from torchcam.utils import overlay_mask
from torchvision.transforms import ToPILImage
import matplotlib.pyplot as plt
from classifiers.only_cnn_classifier import BasicCNNClassifier
import random
import os

model = BasicCNNClassifier()
model.load_state_dict(torch.load("../models/canonical/cnn_only_phishing_classifier.pt"))
model.eval()

dataset_path = os.path.expanduser("~/transfer/phishing_output.h5")

# Here, we use the last convolutional layer in ResNet18, which is 'layer4'
cam_extractor = SmoothGradCAMpp(model.cnn, target_layer="layer4")

dataset = PhishingDataset(required_data=['image', 'url'], split='test', local_file_path=dataset_path)

def get_random_image(dataset):
    random_index = random.randint(0, len(dataset) - 1)
    
    sample = dataset[random_index]    
    tensor_image = sample['image']  
    
    return tensor_image, random_index

def generate_heatmap(image_tensor, item_idx=None):
    image_tensor.requires_grad = True
    
    logits = model(image_tensor.unsqueeze(0)) 
    
    activation_map = cam_extractor(logits.argmax().item(), logits)
    
    to_pil = ToPILImage()
    
    screenshot_image = dataset.screenshots[item_idx]
    result = overlay_mask(to_pil(screenshot_image), to_pil(activation_map[0]), alpha=0.5)
    
    
    plt.figure(figsize=(341, 226))
    plt.imshow(screenshot_image)
    plt.title("Original Image")
    plt.axis('off')
    plt.savefig('original_image.png', bbox_inches='tight')  # Save as a file
    plt.show()
    
    # Save the heatmap overlay image
    plt.figure(figsize=(341, 226))
    plt.imshow(result)
    plt.title("Heatmap Overlay")
    plt.axis('off')
    plt.savefig('heatmap_overlay.png', bbox_inches='tight')  # Save as a file
    plt.show()

# Example usage with a sample image
tensor_image, class_idx = get_random_image(dataset)
print(dataset.labels[class_idx])
generate_heatmap(tensor_image, class_idx)


/opt/homebrew/Caskroom/miniforge/base/envs/phishing-edge/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'dlopen(/opt/homebrew/Caskroom/miniforge/base/envs/phishing-edge/lib/python3.10/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN3c1017RegisterOperatorsD1Ev
  Referenced from: <6DFB383A-E1D9-3EC6-8A60-382AF4E3C226> /opt/homebrew/Caskroom/miniforge/base/envs/phishing-edge/lib/python3.10/site-packages/torchvision/image.so
  Expected in:     <CC2A0259-414A-3562-95F8-DB0DE0A75BD7> /opt/homebrew/Caskroom/miniforge/base/envs/phishing-edge/lib/python3.10/site-packages/torch/lib/libtorch_cpu.dylib'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
Exception ignored in: <bound method IPythonKernel._clean_thre

ImportError: cannot import name 'PhishingClassifier' from 'classifiers.only_cnn_classifier' (/Users/imack/Documents/Stanford/CS230/phishing-edge/classifiers/only_cnn_classifier.py)